In [56]:
import sys,re,csv
sys.path.append('../')
from collections import defaultdict
outputfolder = "/Users/amycweng/DH/SERMONS_APP/db/data"

In [64]:
import re
bible = {}
with open(f'../assets/bible/bible_parts.txt','r') as file:
    lines = file.readlines()
    for idx, line in enumerate(lines):
      if len(re.findall("-",line)) >= 2 and not re.search(" ", line.strip("\n")):
        verse = line.strip("\n")
        text = lines[idx+1].strip("\n")
        if verse not in bible: bible[verse] = []
        bible[verse].append(text)

In [65]:
unique_bible = {}
vidx = -1 
for label, v_list in bible.items():
    for verse in v_list:
        if verse not in unique_bible:
            vidx += 1 
            unique_bible[vidx] = (verse, [])
        unique_bible[vidx][1].append(label)

In [70]:
phrase_to_idx = {}
phrases_csv = []
for vidx, vtuple in unique_bible.items(): 
    phrase_to_idx[vtuple[0]] = vidx
    phrases_csv.append({"vidx":vidx, "phrase":unique_bible[vidx][0]})

In [81]:
bible_vindices = []
for label, vlist in bible.items(): 
    label = re.sub("-"," ",label).split(" ")
    label = " ".join(label[:-2]) + " " + label[-2] + ":" + label[-1] + " (KJV)"
    for v in vlist: 
        vidx = phrase_to_idx[v]
        bible_vindices.append({'vidx':vidx, 'verse_id':label})

In [82]:
with open(f"{outputfolder}/bible_phrase_indices.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=bible_vindices[0].keys())
    writer.writerows(bible_vindices)

In [71]:
with open(f"{outputfolder}/bible_phrases.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=phrases_csv[0].keys())
    writer.writerows(phrases_csv)

In [90]:
verse_hits = defaultdict(list)
with open('../assets/quotes_paraphrases/pre-Elizabethan.txt','r') as file:
    lines = file.readlines()
    for idx, line in enumerate(lines): 
        if len(re.findall("-",line)) >= 2: 
            phrase = lines[idx+1].strip("\n")
            vidx = phrase_to_idx[phrase]
        elif re.search("\t",line): 
            score, meta, doc = line.strip("\n").split("\t")[1:]
            score = float(score)
            meta = meta.strip("(|)").split(", ")
            tcpID = meta[-1].strip("'")
            sidx = meta[0]
            if meta[1] == '-1': loc = "In-Text" 
            else: loc = f"Note {meta[1]}"
            verse_hits[(tcpID,sidx)].append((score, vidx, loc))
hit_csv = []
for meta, hit_list in verse_hits.items(): 
    for h in hit_list: 
        hit_csv.append({"tcpID":meta[0], "sidx": meta[1], 'loc':h[2],
                        "vidx":h[1] , "score":h[0]})

In [91]:
with open(f"{outputfolder}/possible_quotes_paraphrases.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=hit_csv[0].keys())
    writer.writerows(hit_csv)